# 1.3 신경망의 학습

## 1.3.1 Classification 의 Softmax and Cross-Entropy Error

#### Softmax
- score값을 확률값으로 나타냄.
$$y_{k} = exp(S_{k})/\sum_{i=1}^{n}(s_{i})$$
- 발산의 위험이 있기 때문에 Max 값을 뺀 수식을 사용하기도 함
$$y_{k} = exp(S_{k}-S_{max})/\sum_{i=1}^{n}(S_{i}-S_{max})$$

#### Entropy
- entropy : 데이터 안에서의 불균형성을 나타내는 지표
$$L = -\sum_{k}t_{k}logt_{k}$$

#### Cross-Entropy Error
- Cross-Entropy Error
    - entropy에서 비롯되어 두 데이터의 불균형, 두 데이터 간의 차이를 나타냄.
    - 해당 확률분포를 통한 전략을 취했을 때, 더 작은 값. 따라서
    - 원래의 확률분포와 비슷해질 수록 그 값이 감소 -> 비용함수로 자주 사용
        - 하나의 데이터
$$L = -\sum_{k}t_{k}logy_{k}$$
        - 여러개의 데이터
$$L = -\frac{1}{n}\sum_{n}\sum_{k}t_{nk}logy_{nk}$$

## 1.3.2 미분과 기울기
- L에 대한 벡터/행렬의 기울기는 그 벡터/행렬의 형상과 같음

## 1.3.3 연쇄 법칙
- 오차역전파법(back-propagation)
    - chain rule을 적용한다면 국소적인 부분에만 집중함으로써 미분값을 구하기 때문에 아무리 많은 함수를 연결하더라도 미분값을 구할 수 있음

## 1.3.4 계산 그래프
- 덧셈
- 곱셈
- 분기 -> repeat

In [2]:
# repeat 노드
import numpy as np
D, N = 8,7
x = np.random.randn(1, D)
y = np.repeat(x, N, axis=0)

dy = np.random.randn(N, D) # 무작위 기울기
dx = np.sum(dy, axis=0, keepdims=True) # 역전파

In [6]:
# sum 노드
import numpy as np
D, N = 8,7
x = np.random.randn(N,D) # 입력
y = np.sum(x, axis=0, keepdims=True) # 순전파

dy = np.random.randn(1,D) # 무작위 기울기
dx = np.repeat(dy, N, axis=0) # 역전파

array([[-2.05181639,  0.08011393,  1.9892729 ,  3.41522124, -0.32573228,
        -0.35432344,  2.8137993 , -2.91847634]])

In [7]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW # 깊은 복사 - 값이 바뀌는 것을 방지해주기 위해서
        return dx

## 1.3.5 기울기 도출과 역전파 구현

In [8]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [],[]
        self.out = None
        
    def forward(self,x):
        out = 1/(1+np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.-self.out)*self.out
        return dx

In [9]:
class Affine:
    def __init__(self, W, b):
        self.params = [W,b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x,W)+b
        self.x = x
        return out
    
    def backward(self,dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        
        return dx

## 1.3.6 가중치 갱신

In [10]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]